In [ ]:
!pip install crawl4ai requests psutil nest_asyncio
!apt install chromium-browser
!playwright install chromium

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-browser is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [ ]:
"""
3-crawl_docs_FAST.py
--------------------
Batch-crawls a list of documentation URLs in parallel using Crawl4AI's arun_many and a memory-adaptive dispatcher.
Tracks memory usage, prints a summary of successes/failures, and is suitable for large-scale doc scraping jobs.
Usage: Call main() or run as a script. Adjust max_concurrent for parallelism.
"""
import os
import sys
import psutil
import asyncio
import requests
from typing import List
from xml.etree import ElementTree
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode, MemoryAdaptiveDispatcher

async def crawl_parallel(urls: List[str], max_concurrent: int = 10):
    print("\n=== Parallel Crawling with arun_many + Dispatcher ===")

    # Track the peak memory usage for observability
    peak_memory = 0
    process = psutil.Process(os.getpid())
    def log_memory(prefix: str = ""):
        nonlocal peak_memory
        current_mem = process.memory_info().rss  # in bytes
        if current_mem > peak_memory:
            peak_memory = current_mem
        print(f"{prefix} Current Memory: {current_mem // (1024 * 1024)} MB, Peak: {peak_memory // (1024 * 1024)} MB")

    # Configure the browser for headless operation and resource limits
    browser_config = BrowserConfig(
        headless=True,
        verbose=False,
        extra_args=["--disable-gpu", "--disable-dev-shm-usage", "--no-sandbox"],
    )
    # Set up crawl config and dispatcher for batch crawling
    crawl_config = CrawlerRunConfig(cache_mode=CacheMode.BYPASS, stream=False)
    dispatcher = MemoryAdaptiveDispatcher(
        memory_threshold_percent=70.0,  # Don't exceed 70% memory usage
        check_interval=1.0,             # Check memory every second
        max_session_permit=max_concurrent  # Max parallel browser sessions
    )

    async with AsyncWebCrawler(config=browser_config) as crawler:
        log_memory("Before crawl: ")
        # arun_many handles all URLs in parallel, batching and resource management handled by dispatcher
        results = await crawler.arun_many(
            urls=urls,
            config=crawl_config,
            dispatcher=dispatcher
        )
        success_count = 0
        fail_count = 0
        # Loop through all crawl results and tally success/failure
        scraped_data = []

        for result in results:
            if result.success:
                success_count += 1
                scraped_data.append(f"URL: {result.url}\n\n{result.markdown}\n\n{'='*80}\n\n")
            else:
                print(f"Error crawling {result.url}: {result.error_message}")
                fail_count += 1

        # Save to file after crawling
        with open("scraped_output.txt", "w", encoding="utf-8") as f:
            f.writelines(scraped_data)


        print(f"\nSummary:")
        print(f"  - Successfully crawled: {success_count}")
        print(f"  - Failed: {fail_count}")
        log_memory("After crawl: ")
        print(f"\nPeak memory usage (MB): {peak_memory // (1024 * 1024)}")

def get_website_urls():
    """
    Fetches all URLs from the Pydantic AI documentation.
    Uses the sitemap (https://ai.pydantic.dev/sitemap.xml) to get these URLs.

    Returns:
        List[str]: List of URLs
    """
    sitemap_url = "https://docs.crawl4ai.com/sitemap.xml"
    try:
        response = requests.get(sitemap_url)
        response.raise_for_status()

        # Parse the XML
        root = ElementTree.fromstring(response.content)

        # Extract all URLs from the sitemap
        # The namespace is usually defined in the root element
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        urls = [loc.text for loc in root.findall('.//ns:loc', namespace)]

        return urls
    except Exception as e:
        print(f"Error fetching sitemap: {e}")
        return []

async def main():
    urls = get_website_urls()
    if urls:
        print(f"Found {len(urls)} URLs to crawl")
        await crawl_parallel(urls, max_concurrent=10)
    else:
        print("No URLs found to crawl")

In [ ]:
import nest_asyncio
nest_asyncio.apply()
await main()

Found 52 URLs to crawl

=== Parallel Crawling with arun_many + Dispatcher ===
Before crawl:  Current Memory: 211 MB, Peak: 211 MB


[FETCH]... ↓ https://docs.crawl4ai.com/advanced/crawl-dispatcher/                                                 |
✓ | ⏱: 9.70s 

[SCRAPE].. ◆ https://docs.crawl4ai.com/advanced/crawl-dispatcher/                                                 |
✓ | ⏱: 1.44s 

[COMPLETE] ● https://docs.crawl4ai.com/advanced/crawl-dispatcher/                                                 |
✓ | ⏱: 11.30s 

[FETCH]... ↓ https://docs.crawl4ai.com/                                                                           |
✓ | ⏱: 20.68s 

...

[COMPLETE] ● https://docs.crawl4ai.com/extraction/llm-strategies/                                                 |
✓ | ⏱: 11.21s 

[FETCH]... ↓ https://docs.crawl4ai.com/extraction/no-llm-strategies/                                              |
✓ | ⏱: 11.07s 

[SCRAPE].. ◆ https://docs.crawl4ai.com/extraction/no-llm-strategies/                                              |
✓ | ⏱: 0.12s 

[COMPLETE] ● https://docs.crawl4ai.com/extraction/no-llm-strategies/                                              |
✓ | ⏱: 11.19s 


Summary:
  - Successfully crawled: 52
  - Failed: 0
After crawl:  Current Memory: 232 MB, Peak: 232 MB

Peak memory usage (MB): 232


In [ ]:
from google.colab import files
files.download("scraped_output.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>